In [1]:
from bs4 import BeautifulSoup
import csv
import json
import math
import numpy as np
import pandas as pd
import re
import requests
import collections
import pandas as pd
import os

In [ ]:
# load csv downloaded from https://clinicaltrials.gov
df = pd.read_csv("Data/clinical_trials_orginal/clinicaltrials_data_direct_download.csv")
df

,Rank,NCT Number,Title,Acronym,Status,Study Results,Conditions,Interventions,Outcome Measures,Sponsor/Collaborators,...,Other IDs,Start Date,Primary Completion Date,Completion Date,First Posted,Results First Posted,Last Update Posted,Locations,Study Documents,URL
0,1,NCT02538471,LY2157299 Monohydrate (LY2157299) and Radiothe...,NaN,Terminated,Has Results,Metastatic Breast Cancer,Radiation: Radiation therapy|Drug: Study Drug,Number of Participants With Adverse Events|Num...,Weill Medical College of Cornell University|Un...,...,1505016222,"August 10, 2015","September 4, 2018","February 20, 2019","September 2, 2015","December 9, 2019","December 9, 2019","Weill Cornell Medical College, New York, New Y...","""Study Protocol and Statistical Analysis Plan""...",https://ClinicalTrials.gov/show/NCT02538471
1,2,NCT01596530,Evaluation of Drug Activity in Women With Brea...,CHIVE,Terminated,No Results Available,Breast Neoplasm,Drug: Drug-AZD8931|Drug: Drug-Placebo,Comparison of the effects of AZD8931 versus pl...,AstraZeneca,...,D0102C00019,June 2012,March 2013,May 2013,"May 11, 2012",NaN,"December 17, 2012","Research Site, Dusseldorf, Germany|Research Si...",NaN,https://ClinicalTrials.gov/show/NCT01596530
2,3,NCT04194528,S1916 Digital Medicine Program for Pain Contro...,NaN,Terminated,No Results Available,Metastatic Cancer,Drug: Oxycodone/acetaminophen 5/325 mg|Device:...,"Feasibility determined by accrual, adherence a...",Southwest Oncology Group|Proteus Digital Healt...,...,S1916,"October 15, 2019","September 16, 2020","September 16, 2020","December 11, 2019",NaN,"January 13, 2021","Decatur Memorial Hospital, Decatur, Illinois, ...",NaN,https://ClinicalTrials.gov/show/NCT04194528
3,4,NCT02578459,A Pilot Study to Evaluate the Efficacy of ITDD...,NaN,Withdrawn,No Results Available,Pain Management|Cancer Pain,Device: Intrathecal Drug Delivery System|Other...,Visual Analog Scale Pain Scores|Survival Rates...,Flowonix Medical,...,PM-02,April 2016,December 2016,September 2018,"October 19, 2015",NaN,"February 15, 2019","Menorah Medical Center, Overland Park, Kansas,...",NaN,https://ClinicalTrials.gov/show/NCT02578459
4,5,NCT01987375,Cetuximab IRDye800 Study as an Optical Imaging...,NaN,Terminated,No Results Available,Head and Neck Cancer,Drug: Cetuximab-IRDye800,Number of subjects in a cohort experiencing a ...,Eben Rosenthal|Stanford University,...,IRB-35068|ENT0049,November 2015,"April 19, 2017","April 19, 2017","November 19, 2013",NaN,"June 24, 2019","Stanford University, School of Medicine, Stanf...",NaN,https://ClinicalTrials.gov/show/NCT01987375
5,6,NCT04281290,Electrochemotherapy of Posterior Resection Sur...,PanECT,Terminated,No Results Available,Pancreas Cancer,Procedure: Electrochemotherapy - electroporati...,Evaluation of feasibility and safety|Disease-f...,University Medical Centre Ljubljana|Institute ...,...,21K/01/20,"July 13, 2020","March 9, 2021","March 9, 2021","February 24, 2020",NaN,"March 12, 2021","University Medical Centre Ljubljana, Ljubljana...",NaN,https://ClinicalTrials.gov/show/NCT04281290
6,7,NCT02244112,A Phase 1 Study of Oprozomib to Assess Food Ef...,NaN,Terminated,No Results Available,Advanced Non-Central Nervous System (CNS) Mali...,Drug: Oprozomib|Drug: Midazolam,Food Effect/QTc - Cmax|Food Effect/QTc - AUC|F...,Amgen,...,OPZ009,August 2014,"May 15, 2015","July 10, 2019","September 18, 2014",NaN,"February 21, 2021",University of Colorado Anschutz Medical Campus...,NaN,https://ClinicalTrials.gov/show/NCT02244112
7,8,NCT03370302,"A Study to Evaluate the Safety, Tolerability, ...",NaN,Terminated,Has Results,Advanced Nonhematological Neoplasms,Drug: TAK-228,Number of Participants With Treatment Emergent...,"Millennium Pharmaceuticals, Inc.|Takeda",...,C31008|U1111-1202-4296|1066064639|20170270241|...,"January 17, 2018","August 28, 2019","August 28, 2019","December 12, 2017","September 9, 2020","May 11, 2021","National Cancer Center Hospital East, Kashiwa-...","""Study Protocol"", https://ClinicalTrials.gov/P...",

In [ ]:
# scrape recruitment status details(reason for inactivity/termination)
details = []
start = 0
end = 2000
i=0
for url in df['URL'][start:end]: 
    response1 = requests.get(url)
    soup1 = BeautifulSoup(response1.content, "lxml")
    recruitment_status = soup1.find("div",class_="tr-status tr-not-recruiting-colors").get_text().split('\n')
    details.append(recruitment_status[5].strip()[1:-1])
    i=i+1
    print(i) #know how many are scraped

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


1861
1862
1863
1864
1865
1866
1867
1868
1869
1870
1871
1872
1873
1874
1875
1876
1877
1878
1879
1880
1881
1882
1883
1884
1885
1886
1887
1888
1889
1890
1891
1892
1893
1894
1895
1896
1897
1898
1899
1900
1901
1902
1903
1904
1905
1906
1907
1908
1909
1910
1911
1912
1913
1914
1915
1916
1917
1918
1919
1920
1921
1922
1923
1924
1925
1926
1927
1928
1929
1930
1931
1932
1933
1934
1935
1936
1937
1938
1939
1940
1941
1942
1943
1944
1945
1946
1947
1948
1949
1950
1951
1952
1953
1954
1955
1956
1957
1958
1959
1960
1961
1962
1963
1964
1965
1966
1967
1968
1969
1970
1971
1972
1973
1974
1975
1976
1977
1978
1979
1980
1981
1982
1983
1984
1985
1986
1987
1988
1989
1990
1991
1992
1993
1994
1995
1996
1997
1998
1999
2000


In [ ]:
# update df
df["Reason for Inactivity"] = ""
df["Reason for Inactivity"][start:end] = details
df

/opt/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/opt/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


,Rank,NCT Number,Title,Acronym,Status,Study Results,Conditions,Interventions,Outcome Measures,Sponsor/Collaborators,...,Primary Completion Date,Completion Date,First Posted,Results First Posted,Last Update Posted,Locations,Study Documents,URL,Recruitment Status,Reason for Inactivity
0,1,NCT02538471,LY2157299 Monohydrate (LY2157299) and Radiothe...,NaN,Terminated,Has Results,Metastatic Breast Cancer,Radiation: Radiation therapy|Drug: Study Drug,Number of Participants With Adverse Events|Num...,Weill Medical College of Cornell University|Un...,...,"September 4, 2018","February 20, 2019","September 2, 2015","December 9, 2019","December 9, 2019","Weill Cornell Medical College, New York, New Y...","""Study Protocol and Statistical Analysis Plan""...",https://ClinicalTrials.gov/show/NCT02538471,Terminated,The study was terminated due to lack of patien...
1,2,NCT01596530,Evaluation of Drug Activity in Women With Brea...,CHIVE,Terminated,No Results Available,Breast Neoplasm,Drug: Drug-AZD8931|Drug: Drug-Placebo,Comparison of the effects of AZD8931 versus pl...,AstraZeneca,...,March 2013,May 2013,"May 11, 2012",NaN,"December 17, 2012","Research Site, Dusseldorf, Germany|Research Si...",NaN,https://ClinicalTrials.gov/show/NCT01596530,Terminated,All AstraZeneca sponsored clinical trials of A...
2,3,NCT04194528,S1916 Digital Medicine Program for Pain Contro...,NaN,Terminated,No Results Available,Metastatic Cancer,Drug: Oxycodone/acetaminophen 5/325 mg|Device:...,"Feasibility determined by accrual, adherence a...",Southwest Oncology Group|Proteus Digital Healt...,...,"September 16, 2020","September 16, 2020","December 11, 2019",NaN,"January 13, 2021","Decatur Memorial Hospital, Decatur, Illinois, ...",NaN,https://ClinicalTrials.gov/show/NCT04194528,Terminated,Study sponsor withdrew support.
3,4,NCT02578459,A Pilot Study to Evaluate the Efficacy of ITDD...,NaN,Withdrawn,No Results Available,Pain Management|Cancer Pain,Device: Intrathecal Drug Delivery System|Other...,Visual Analog Scale Pain Scores|Survival Rates...,Flowonix Medical,...,December 2016,September 2018,"October 19, 2015",NaN,"February 15, 2019","Menorah Medical Center, Overland Park, Kansas,...",NaN,https://ClinicalTrials.gov/show/NCT02578459,Withdrawn,The study was never initiated by sponsor. Pati...
4,5,NCT01987375,Cetuximab IRDye800 Study as an Optical Imaging...,NaN,Terminated,No Results Available,Head and Neck Cancer,Drug: Cetuximab-IRDye800,Number of subjects in a cohort experiencing a ...,Eben Rosenthal|Stanford University,...,"April 19, 2017","April 19, 2017","November 19, 2013",NaN,"June 24, 2019","Stanford University, School of Medicine, Stanf...",NaN,https://ClinicalTrials.gov/show/NCT01987375,Terminated,logistics
5,6,NCT04281290,Electrochemotherapy of Posterior Resection Sur...,PanECT,Terminated,No Results Available,Pancreas Cancer,Procedure: Electrochemotherapy - electroporati...,Evaluation of feasibility and safety|Disease-f...,University Medical Centre Ljubljana|Institute ...,...,"March 9, 2021","March 9, 2021","February 24, 2020",NaN,"March 12, 2021","University Medical Centre Ljubljana, Ljubljana...",NaN,https://ClinicalTrials.gov/show/NCT04281290,Terminated,After enrolling 3 participants the safety and ...
6,7,NCT02244112,A Phase 1 Study of Oprozomib to Assess Food Ef...,NaN,Terminated,No Results Available,Advanced Non-Central Nervous System (CNS) Mali...,Drug: Oprozomib|Drug: Midazolam,Food Effect/QTc - Cmax|Food Effect/QTc - AUC|F...,Amgen,...,"May 15, 2015","July 10, 2019","September 18, 2014",NaN,"February 21, 2021",University of Colorado Anschutz Medical Campus...,NaN,https://ClinicalTrials.gov/show/NCT02244112,Terminated,"Why study stopped revised to ""A program evalua..."
7,8,NCT03370302,"A Study to Evaluate the Safety, Tolerability, ...",NaN,Terminated,Has Results,Advanced Nonhematological Neoplasms,Drug: TAK-228,Number of Participants With Treatment Emergent...,"Millennium Pharmaceuticals, Inc.|Takeda",...,"August 28, 2019","August 28, 2019","December

In [ ]:
# convert to csv 
csv_name = "clinicaltrials_data_"+str(start)+"_"+str(end)+".csv"
path = "Data/clinical_trials_orginal/"
df.to_csv(os.path.join(path,csv_name),index=False)